In [1]:
%store -r train 
%store -r test

# Импорт

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

In [3]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

Python       : 3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]
Numpy        : 1.22.2


In [4]:
#Удаление дублирующихся строк
train_mod = train.drop_duplicates()
test_mod = test.drop_duplicates()

print('TRAIN','Rows =', np.shape(train_mod)[0],'Columns =', np.shape(train_mod)[1], sep=' ')
print('TEST ','Rows =', np.shape(test_mod)[0],'Columns =', np.shape(test_mod)[1], sep=' ') 

TRAIN Rows = 87264 Columns = 26
TEST  Rows = 34686 Columns = 32


# Функции

In [5]:
#Словарь соответствий брендов странас происхождения
countries_dict = {'Germany': ['AUDI','BMW','OPEL','PORSCHE','VOLKSWAGEN','MERCEDES'],
                  'Japan':   ['HONDA','INFINITI','LEXUS','MAZDA','MITSUBISHI','NISSAN','SUBARU','SUZUKI','TOYOTA'],
                  'USA':     ['CADILLAC','CHEVROLET','CHRYSLER','DODGE','FORD','JEEP'],
                  'China':   ['CHERY','GEELY','GREAT_WALL'],
                  'UK':      ['JAGUAR','MINI','LAND_ROVER'],
                  'Korea':   ['DAEWOO','HYUNDAI','KIA','SSANG_YONG'],
                  'Sweden':  ['VOLVO'],
                  'Czech':   ['SKODA']}

In [6]:
#Функция для заполнения столбца со страной происхождения бренда
def get_country(x):
    if x in countries_dict['Germany']:
        x = 'Germany'
    elif x in countries_dict['Japan']:
        x = 'Japan'
    elif x in countries_dict['USA']:
        x = 'USA'
    elif x in countries_dict['China']:
        x = 'China'
    elif x in countries_dict['UK']:
        x = 'UK'
    elif x in countries_dict['Korea']:
        x = 'Korea'
    elif x in countries_dict['Sweden']:
        x = 'Sweden'
    else:
        x = 'Czech'
    return x
#Наверное, эта функция должна выглядеть не так колхозно


#Функция для просмотра и сравнения типов данных в столбцах
def type_comparison(column):
    print('TEST')
    display(test_mod[column].sample(5))
    print('TRAIN')
    display(train_mod[column].sample(5))  
    
    
    
#Функция для приведения значений столбца bodyType к единому формату
def get_bodyType(x):
    row = str(x).lower()
    bodyType = re.findall('[а-яё]+', row)
    if bodyType == []:
        return None
    return str(bodyType[0])



#Функция для приведения значений столбца engineDisplacement к единому формату
def get_EngineDisplacement(x):
    row = str(x)
    engine = re.findall('\d\.\d', row)
    if engine == []:
        return None
    return float(engine[0])



#Функция для извлечения значения объёма двигателя из столбца name
def pull_EngineDisplacement(x):
    row = str(x)
    engine = re.findall('\d\.\d', row)
    if len(engine) !=0:
        return float(engine[0])
    else:
        return 0

    
    
#Список имён столбцов под удаление. Будет заполняться по ходу первичного анализа.
columns_to_drop = []

In [7]:
'''Общая предобработка должна сделаться для двух датасетов'''
#Добавление к обоим датасетам столбца 'TEST' и заполнение его единицами в test_mod и нолями в train_mod
#test_mod['TEST'] = 1
#train_mod['TEST'] = 0

#Объединение датасетов
#whole_data = train_mod.append(test_mod)
''''''




#Создание столбца 'Country_of_origin' со страной происхождения бренда
test_mod['Country_of_origin'] = test_mod['brand'].apply(get_country)
train_mod['Country_of_origin'] = train_mod['brand'].apply(get_country)

#Очистка столбца 'bodyType' и приведение к единому формату
test_mod['bodyType'] = test_mod.bodyType.apply(get_bodyType)
train_mod['bodyType'] = train_mod.bodyType.apply(get_bodyType)

#Приведение столбца 'color' к единому формату
train_mod = train_mod.replace({'color':
                            {'040001': 'чёрный',
                            'EE1D19': 'красный',
                            '0000CC': 'синий',
                            'CACECB': 'серебристый',
                            '007F00': 'зелёный',
                            'FAFBFB': 'белый',
                            '97948F': 'серый',
                            '22A0F8': 'голубой',
                            '660099': 'фиолетовый',
                            '200204': 'коричневый',
                            'C49648': 'бежевый',
                            'DEA522': 'золотистый',
                            '4A2197': 'пурпурный',
                            'FFD600': 'жёлтый',
                            'FF8649': 'оранжевый',
                            'FFC0CB': 'розовый'}})

#Приведение столбца 'engineDisplacement' к единому формату 
train_mod['engineDisplacement'] = train_mod.engineDisplacement.apply(get_EngineDisplacement)
test_mod['engineDisplacement'] = test_mod.engineDisplacement.apply(get_EngineDisplacement)

#Заполнение пропусков в столбце 'engineDisplacement' датасета test_mod нолями
test_mod['engineDisplacement'].fillna(0, inplace = True)

#Приведение столбца 'enginePower' к единому формату
train_mod.enginePower = train_mod.enginePower.convert_dtypes(int)
test_mod['enginePower'] = test_mod['enginePower'].apply(lambda x : (int)(x.split(' ')[0]))

#Удаление из датасета train_mod ряда, содержащего слишком много пропусков и ошибок
train_mod.drop(24624, inplace = True)

#Приведение столбца 'modelDate' к единому формату
train_mod.modelDate = train_mod.modelDate.convert_dtypes(int)

#Извлечение значений объёма двигателя из столбца 'name' для заполнения пропусков в столбце 'engineDisplacement'
#в датасете train_mod
train_mod['engineDisplacement'] = train_mod['name'].apply(pull_EngineDisplacement)

#Добавление столбца 'name' в список столбцов на удаление 
#(вся информация, содержащаяся в нём, дублируется в других столбцах)
columns_to_drop.append('name')

#Приведение столбца 'numberOfDoors' к единому формату
train_mod.numberOfDoors = train_mod.numberOfDoors.convert_dtypes(int)

#Добавление столбца 'vehicleConfiguration' в список столбцов на удаление 
#(вся информация, содержащаяся в нём, дублируется в других столбцах)
columns_to_drop.append('vehicleConfiguration')

#Приведение столбца 'vehicleTransmission' к единому формату
test_mod = test_mod.replace({'vehicleTransmission':
                            {'автоматическая': 'AUTOMATIC',
                            'вариатор': 'VARIATOR',
                            'механическая': 'MECHANICAL',
                            'роботизированная': 'ROBOT'}})

#Приведение столбца 'Владельцы' к единому формату
train_mod.Владельцы = train_mod.Владельцы.convert_dtypes(int)

test_mod = test_mod.replace({'Владельцы':
                            {'1\xa0владелец': 1,
                            '2\xa0владельца': 2,
                            '3 или более': 3}})

#Заполнение пропусков в столбце 'Владельцы' датасета train_mod нолями
train_mod['Владельцы'].fillna(0, inplace = True)

#Добавление столбца 'vehicleConfiguration' в список столбцов на удаление 
#(Более 60% пропусков)
columns_to_drop.append('Владение')

#Приведение столбца 'ПТС' к единому формату
test_mod = test_mod.replace({'ПТС':
                            {'Оригинал': 'ORIGINAL',
                            'Дубликат': 'DUPLICATE'}})

#Приведение столбца 'Руль' к единому формату
test_mod = test_mod.replace({'Руль':
                            {'Левый': 'LEFT',
                            'Правый': 'RIGHT'}})

#Добавление столбца 'Состояние' в список столбцов на удаление 
#(В test_mod только одно уникальное значение, в train_mod - пустой столбец)
columns_to_drop.append('Состояние')

#Добавление столбца 'Таможня' в список столбцов на удаление 
#(Одно уникальное значение в обоих датасетах)
columns_to_drop.append('Таможня')

#Удаление общис столбцов по списку
test_mod.drop(columns = columns_to_drop, inplace = True)
train_mod.drop(columns = columns_to_drop, inplace = True)

#Удаление лишних столбцов для приведения датасетов к единому виду 
test_drop = []
for i in range(len(test_mod.columns)):
    if test_mod.columns[i] not in train_mod.columns:
        test_drop.append(test_mod.columns[i])

test_mod.drop(columns = test_drop, inplace = True)


train_drop = []
for i in range(len(train_mod.columns)):
    if train_mod.columns[i] not in test_mod.columns:
        train_drop.append(train_mod.columns[i])
        
train_mod.drop(columns = train_drop, inplace = True)



C:\Users\zetet\AppData\Local\Temp\ipykernel_18552\4675327.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_mod['Country_of_origin'] = train_mod['brand'].apply(get_country)
C:\Users\zetet\AppData\Local\Temp\ipykernel_18552\4675327.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_mod['bodyType'] = train_mod.bodyType.apply(get_bodyType)
